In [ ]:
# pip install pulp pandas numpy matplotlib scipy scikit-learn seaborn

Aqui está a versão refinada do texto, com maior precisão técnica e concisão:

**Formulação Matemática do PCV:**

A função objetivo minimiza o custo total do circuito:  
$$
\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} x_{ij}
$$  
onde:  
- $ c_{ij} $ representa o custo/distância entre as cidades $ i $ e $ j $  
- $ x_{ij} $ é a variável binária que indica se o arco $ (i,j) $ faz parte da rota  

**Restrições essenciais:**  
1. **Visita única a cada nó:**  
   $$
   \sum_{i=1}^{n} x_{ij} = 1 \quad \forall i \in \{1,...,n\}
   $$

   e

   $$
   \sum_{j=1}^{n} x_{ij} = 1 \quad \forall j \in \{1,...,n\}
   $$
2. **Eliminação de sub-rotas:**  
   $$
   \sum_{i \in S} \sum_{j \in S} x_{ij} \leq |S| - 1 \quad \forall S \subset N, 2 \leq |S| \leq n-1
   $$  
3. **Variáveis binárias:**  
   $$
   x_{ij} \in \{0,1\} \quad \forall (i,j)
   $$

In [ ]:
import numpy as np
import classes.ClassCaixeiro as cc

coords = np.array([
    [10, 30],  # 1
    [20, 50],  # 2
    [50, 90],  # 3
    [70, 30],  # 4
    [90, 50],  # 5
])

cc.plot_points(coords, "Cidades (apenas pontos)")


subroutes = [[0, 1, 2, 0], [0, 3, 4, 0]]
for idx, sr in enumerate(subroutes, 1):
    cc.plot_route(coords, sr, f"Sub‑rota {idx}: {[i+1 for i in sr]}")

Abaixo, estão as coordenadas de cada cidade:

| Cidade | $x$ | $y$ |
| ------ | --- | --- |
| 1      | 10  | 30  |
| 2      | 20  | 50  |
| 3      | 50  | 90  |
| 4      | 70  | 30  |
| 5      | 90  | 50  |

---

$$
d(i,j) = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}
$$

---

### 🔢 Cálculos manuais das distâncias principais

#### Entre cidade 1 e 2:

$$
\sqrt{(10 - 20)^2 + (30 - 50)^2} = \sqrt{(-10)^2 + (-20)^2} = \sqrt{100 + 400} = \sqrt{500} \approx \boxed{22.36}
$$

#### Entre cidade 1 e 3:

$$
\sqrt{(10 - 50)^2 + (30 - 90)^2} = \sqrt{(-40)^2 + (-60)^2} = \sqrt{1600 + 3600} = \sqrt{5200} \approx \boxed{72.11}
$$

#### Entre cidade 1 e 4:

$$
\sqrt{(10 - 70)^2 + (30 - 30)^2} = \sqrt{(-60)^2 + 0^2} = \sqrt{3600} = \boxed{60.00}
$$

#### Entre cidade 1 e 5:

$$
\sqrt{(10 - 90)^2 + (30 - 50)^2} = \sqrt{(-80)^2 + (-20)^2} = \sqrt{6400 + 400} = \sqrt{6800} \approx \boxed{82.46}
$$

In [ ]:
print("\nTabela de distâncias euclidianas (unidades):")
cc.distance_table(coords)

#### Variável de decisão

$$
x_{ij}= 
\begin{cases}
1 & \text{se o caixeiro‑viajante vai diretamente da cidade } i \text{ para a cidade } j,\; i\ne j\\
0 & \text{caso contrário}
\end{cases}
\qquad i,j\in N=\{1,2,3,4,5\}
$$

---

#### Função‑objetivo

Minimizar a distância total percorrida:

$$
\boxed{\;F_{\text{obj}}=\min z=\sum_{i=1}^{5}\sum_{\substack{j=1\\[2pt] j\ne i}}^{5}c_{ij}\,x_{ij}\;}
$$

onde $c_{ij}$ é a distância euclidiana entre as cidades $i$ e $j$ (ver tabela gerada no código).

---

### Restrições

1. **Cada nó deve ter exatamente uma aresta saindo**

   $$
   \begin{aligned}
   x_{12}+x_{13}+x_{14}+x_{15} &=1& (i=1)\\
   x_{21}+x_{23}+x_{24}+x_{25} &=1& (i=2)\\
   x_{31}+x_{32}+x_{34}+x_{35} &=1& (i=3)\\
   x_{41}+x_{42}+x_{43}+x_{45} &=1& (i=4)\\
   x_{51}+x_{52}+x_{53}+x_{54} &=1& (i=5)
   \end{aligned}
   $$

2. **Cada nó deve ter exatamente uma aresta entrando**

   $$
   \begin{aligned}
   x_{21}+x_{31}+x_{41}+x_{51} &=1& (j=1)\\
   x_{12}+x_{32}+x_{42}+x_{52} &=1& (j=2)\\
   x_{13}+x_{23}+x_{43}+x_{53} &=1& (j=3)\\
   x_{14}+x_{24}+x_{34}+x_{54} &=1& (j=4)\\
   x_{15}+x_{25}+x_{35}+x_{45} &=1& (j=5)
   \end{aligned}
   $$

3. **Eliminação de subtours com 2 nós ($|S|=2$)**

   $$
   \begin{aligned}
   x_{12}+x_{21}&\le 1\\
   x_{13}+x_{31}&\le 1\\
   x_{14}+x_{41}&\le 1\\
   x_{15}+x_{51}&\le 1\\
   x_{23}+x_{32}&\le 1\\
   x_{24}+x_{42}&\le 1\\
   x_{25}+x_{52}&\le 1\\
   x_{34}+x_{43}&\le 1\\
   x_{35}+x_{53}&\le 1\\
   x_{45}+x_{54}&\le 1
   \end{aligned}
   $$

4. **Eliminação de subtours com 3 nós ($|S|=3$)**
   (listar todas as combinações de três índices; cada desigualdade soma as 6 arestas internas e limita a ≤ 2)

   $$
   \begin{aligned}
   x_{12}+x_{23}+x_{31}&\le 2\\
   x_{12}+x_{24}+x_{41}&\le 2\\
   x_{12}+x_{25}+x_{51}&\le 2\\
   x_{13}+x_{34}+x_{41}&\le 2\\
   x_{13}+x_{35}+x_{51}&\le 2\\
   x_{14}+x_{45}+x_{51}&\le 2\\
   x_{23}+x_{35}+x_{52}&\le 2\\
   x_{24}+x_{45}+x_{52}&\le 2\\
   x_{25}+x_{53}+x_{32}&\le 2\\
   x_{34}+x_{45}+x_{53}&\le 2
   \end{aligned}
   $$

5. **Eliminação de subtours com 4 nós ($|S|=4$)**
   (cada desigualdade soma as 12 arestas internas e limita a ≤ 3)

   $$
   \begin{aligned}
   x_{12}+x_{23}+x_{34}+x_{41}&\le 3\\
   x_{12}+x_{23}+x_{35}+x_{51}&\le 3\\
   x_{12}+x_{24}+x_{45}+x_{51}&\le 3\\
   x_{13}+x_{34}+x_{45}+x_{51}&\le 3\\
   x_{23}+x_{35}+x_{45}+x_{52}&\le 3
   \end{aligned}
   $$

6. **Domínio das variáveis**

   $$
   x_{ij}\in\{0,1\},\qquad i,j=1,\dots,5,\; i\ne j
   $$

In [ ]:
dist = cc.euclidean_matrix(coords)
tour, cost = cc.solve_tsp_mtz(dist)
route_str = " → ".join(str(i+1) for i in tour)
print("Rota ótima:", route_str)
print(f"Distância total: {cost:.2f}\n")

# 4.1 Mostrar coordenadas
print("Coordenadas das cidades (x, y):")
for i, (x, y) in enumerate(coords, 1):
    print(f"  Cidade {i}: ({x}, {y})")

In [ ]:
cc.plot_route(coords, tour, f"Rota ótima – custo {cost:.2f}")

In [ ]:
# Novas coordenadas  ───────────────
coords = np.array([
    [ 8, 20],   # 1
    [15, 55],   # 2
    [40, 80],   # 3
    [55, 15],   # 4
    [75, 40],   # 5
    [90, 70],   # 6
    [60, 95],   # 7
    [25, 10],   # 8
    [45, 50],   # 9
    [85, 15],   #10
])

# 1) imprime coordenadas e matriz de distâncias
cc.plot_points(coords, "Novas 10 cidades")     # opcional
print("\n┌── Coordenadas (x, y) ───────────")
for i,(x,y) in enumerate(coords,1):
    print(f"│ Cidade {i}: ({x}, {y})")
print("└────────────────────────────────\n")

dist = cc.euclidean_matrix(coords)

# 2) resolve TSP
tour, cost = cc.solve_tsp_mtz(dist)
route_txt = " → ".join(str(i+1) for i in tour)
print(f"\nRota ótima: {route_txt}")
print(f"Distância total: {cost:.2f}")


In [ ]:

# 3) plota rota ótima (opcional)
cc.plot_route(coords, tour, f"Rota ótima (10 nós) – custo {cost:.2f}")
